In [1]:
test_input1 = "8A004A801A8002F478"

In [148]:
hex_map = {
    '0': '0000',
    '1': '0001',
    '2': '0010',
    '3': '0011',
    '4': '0100',
    '5': '0101',
    '6': '0110',
    '7': '0111',
    '8': '1000',
    '9': '1001',
    'A': '1010',
    'B': '1011',
    'C': '1100',
    'D': '1101',
    'E': '1110',
    'F': '1111',
}

def hex_to_bin(hex_):
    bin_ = ''.join([hex_map[x] for x in hex_])
    return bin_

In [3]:
def popn(txt, n):
    val = int(txt[:n], 2)
    rest = txt[n:]
    return val, rest

def popn_txt(txt, n):
    val = txt[:n]
    rest = txt[n:]
    return val, rest

In [124]:
def parse_operator(txt):
    print('parse operator', txt)
    length_type, rest = popn(txt, 1)
    this_packet_len = 1
    print('length type', length_type)
    values = []
    if length_type == 0:
        total_length, rest = popn(rest, 15)
        this_packet_len += 15
        print('total length', total_length)
        l = 0
        while l < total_length:
            packet_length, value, rest = parse_packet(rest)
            values.append(value)
            l += packet_length
            print('parsing op l', l)
        this_packet_len += total_length
    else:
        n_packets, rest = popn(rest, 11)
        this_packet_len += 11

        n = 0
        this_packet_len
        while n < n_packets:
            packet_length, value, rest = parse_packet(rest)
            values.append(value)
            this_packet_len += packet_length
            n += 1

    print('operator parsed', this_packet_len, values)
    return this_packet_len, values, rest


            
def parse_literal(txt):
    print('parse literal', txt)
    value_txt = ''
    this_packet_len = 0
    rest = txt
    while True:
        this_packet_len += 5
        group_type, rest = popn(rest, 1)
        load, rest = popn_txt(rest, 4)

        value_txt = value_txt + load
        if group_type == 0:
            break
    value = int(value_txt, 2)
    print('literal parsed', this_packet_len, value)
    return this_packet_len, value, rest


type_parse_map = {
    4: parse_literal,
}

        
def parse_packet(txt):
    version, rest =  popn(txt, 3)
    type_, rest = popn(rest, 3)
    print('\nparse', txt)
    print('version', version, 'type', type_)

    parser = type_parse_map.get(type_, parse_operator)
    packet_length, value, rest = parser(rest)
    length = 6 + packet_length
    return length, (type_, version, value), rest

In [125]:
parse_packet('110100101111111000101000111')


parse 110100101111111000101000111
version 6 type 4
parse literal 101111111000101000111
literal parsed 15 2021


(21, (4, 6, 2021), '000111')

In [126]:
parse_packet('00111000000000000110111101000101001010010001001000000000')


parse 00111000000000000110111101000101001010010001001000000000
version 1 type 6
parse operator 00000000000110111101000101001010010001001000000000
length type 0
total length 27

parse 1101000101001010010001001000000000
version 6 type 4
parse literal 0101001010010001001000000000
literal parsed 5 10
parsing op l 11

parse 01010010001001000000000
version 2 type 4
parse literal 10001001000000000
literal parsed 10 20
parsing op l 27
operator parsed 43 [(4, 6, 10), (4, 2, 20)]


(49, (6, 1, [(4, 6, 10), (4, 2, 20)]), '0000000')

In [127]:
parse_packet('11101110000000001101010000001100100000100011000001100000')


parse 11101110000000001101010000001100100000100011000001100000
version 7 type 3
parse operator 10000000001101010000001100100000100011000001100000
length type 1

parse 01010000001100100000100011000001100000
version 2 type 4
parse literal 00001100100000100011000001100000
literal parsed 5 1

parse 100100000100011000001100000
version 4 type 4
parse literal 000100011000001100000
literal parsed 5 2

parse 0011000001100000
version 1 type 4
parse literal 0001100000
literal parsed 5 3
operator parsed 45 [(4, 2, 1), (4, 4, 2), (4, 1, 3)]


(51, (3, 7, [(4, 2, 1), (4, 4, 2), (4, 1, 3)]), '00000')

In [128]:
parse_packet(hex_to_bin('8A004A801A8002F478'))


parse 100010100000000001001010100000000001101010000000000000101111010001111000
version 4 type 2
parse operator 100000000001001010100000000001101010000000000000101111010001111000
length type 1

parse 001010100000000001101010000000000000101111010001111000
version 1 type 2
parse operator 100000000001101010000000000000101111010001111000
length type 1

parse 101010000000000000101111010001111000
version 5 type 2
parse operator 000000000000101111010001111000
length type 0
total length 11

parse 11010001111000
version 6 type 4
parse literal 01111000
literal parsed 5 15
parsing op l 11
operator parsed 27 [(4, 6, 15)]
operator parsed 45 [(2, 5, [(4, 6, 15)])]
operator parsed 63 [(2, 1, [(2, 5, [(4, 6, 15)])])]


(69, (2, 4, [(2, 1, [(2, 5, [(4, 6, 15)])])]), '000')

In [129]:
def sum_version(code):
    type_, version, values = code
    s = version
    if type(values) is not list:
        return version
    for v in values:
        s += sum_version(v)
    return s


In [138]:
cases = [
    '8A004A801A8002F478',
    '620080001611562C8802118E34',
    'C0015000016115A2E0802F182340',
    'A0016C880162017C3686B18A3D4780',
]

values = []
for c in cases:
    _, code, _ = parse_packet(hex_to_bin(c))
    s = sum_version(code)
    values.append(s)


parse 100010100000000001001010100000000001101010000000000000101111010001111000
version 4 type 2
parse operator 100000000001001010100000000001101010000000000000101111010001111000
length type 1

parse 001010100000000001101010000000000000101111010001111000
version 1 type 2
parse operator 100000000001101010000000000000101111010001111000
length type 1

parse 101010000000000000101111010001111000
version 5 type 2
parse operator 000000000000101111010001111000
length type 0
total length 11

parse 11010001111000
version 6 type 4
parse literal 01111000
literal parsed 5 15
parsing op l 11
operator parsed 27 [(4, 6, 15)]
operator parsed 45 [(2, 5, [(4, 6, 15)])]
operator parsed 63 [(2, 1, [(2, 5, [(4, 6, 15)])])]

parse 01100010000000001000000000000000000101100001000101010110001011001000100000000010000100011000111000110100
version 3 type 0
parse operator 10000000001000000000000000000101100001000101010110001011001000100000000010000100011000111000110100
length type 1

parse 0000000000000000010110000

In [139]:
values

[16, 12, 23, 31]

In [187]:
with open('input.txt', 'r') as f:
    input_ = f.read().strip()
    
_, code, _ = parse_packet(hex_to_bin(input_))


parse 000000100000110101110000100000000100000100100101100011000000101101001100011010000011111001100001111101100111010010100001010000000001010110010101110011000011110111100110011010011010110001001111101101111011111011111110111010000100000000011000001011001101111100000011001101000000000100101001011011110100010000110110011111111001001110000011011100011000000000101101001011001100100110000000000110101001100000000000001000010011000001000110000010011100010000110001000000000111001000111001110000101100100001100000010000000000111101111100001101101011000000000101111001000100011010100100010001100110001010100010100000000101100100100101111111111001011011001011110011100000000000110011110001010111001101101101000100111101100111001111110011011100000000000001110010001001101111110101011101010000010101111001100111000000110100011000000000101101001001100011100110000000000110101001000000000000001000010001000001011010001110100100001111000001000000000111101000011110110000110110100010100111001011011000011000010

In [188]:
sum_version(code)

965

# Part 2

In [180]:
import numpy as np

def greater(v):
    return int(v[0] > v[1])
def less(v):
    return int(v[0] < v[1])
def equal(v):
    return int(v[0] == v[1])

type_to_fun = {
    0: np.sum,
    1: np.prod,
    2: np.min,
    3: np.max,
    5: greater,
    6: less,
    7: equal,
}

In [181]:
_, code, _ = parse_packet(hex_to_bin('C200B40A82'))


parse 1100001000000000101101000000101010000010
version 6 type 0
parse operator 1000000000101101000000101010000010
length type 1

parse 1101000000101010000010
version 6 type 4
parse literal 0000101010000010
literal parsed 5 1

parse 01010000010
version 2 type 4
parse literal 00010
literal parsed 5 2
operator parsed 34 [(4, 6, 1), (4, 2, 2)]


In [182]:
def execute(code):
    type_, version, values = code
    if type_ == 4:
        return values

    fun = type_to_fun[type_]
    args = []
    for v in values:
        arg = execute(v)
        args.append(arg)
    res = fun(args)
    return res

print(code)
execute(code)

(0, 6, [(4, 6, 1), (4, 2, 2)])


3

In [183]:
cases = [
    'C200B40A82',
    '04005AC33890',
    '880086C3E88112',
    'CE00C43D881120',
    'D8005AC2A8F0',
    'F600BC2D8F',
    '9C005AC2F8F0',
    '9C0141080250320F1802104A08',
]


values = []
for c in cases:
    _, code, _ = parse_packet(hex_to_bin(c))
    s = execute(code)
    values.append(s)


parse 1100001000000000101101000000101010000010
version 6 type 0
parse operator 1000000000101101000000101010000010
length type 1

parse 1101000000101010000010
version 6 type 4
parse literal 0000101010000010
literal parsed 5 1

parse 01010000010
version 2 type 4
parse literal 00010
literal parsed 5 2
operator parsed 34 [(4, 6, 1), (4, 2, 2)]

parse 000001000000000001011010110000110011100010010000
version 0 type 1
parse operator 000000000001011010110000110011100010010000
length type 0
total length 22

parse 10110000110011100010010000
version 5 type 4
parse literal 00110011100010010000
literal parsed 5 6
parsing op l 11

parse 011100010010000
version 3 type 4
parse literal 010010000
literal parsed 5 9
parsing op l 22
operator parsed 38 [(4, 5, 6), (4, 3, 9)]

parse 10001000000000001000011011000011111010001000000100010010
version 4 type 2
parse operator 00000000001000011011000011111010001000000100010010
length type 0
total length 33

parse 1011000011111010001000000100010010
version 5 type 

In [184]:
values

[3, 54, 7, 9, 1, 0, 0, 1]

In [189]:
with open('input.txt', 'r') as f:
    input_ = f.read().strip()
    
_, code, _ = parse_packet(hex_to_bin(input_))


parse 000000100000110101110000100000000100000100100101100011000000101101001100011010000011111001100001111101100111010010100001010000000001010110010101110011000011110111100110011010011010110001001111101101111011111011111110111010000100000000011000001011001101111100000011001101000000000100101001011011110100010000110110011111111001001110000011011100011000000000101101001011001100100110000000000110101001100000000000001000010011000001000110000010011100010000110001000000000111001000111001110000101100100001100000010000000000111101111100001101101011000000000101111001000100011010100100010001100110001010100010100000000101100100100101111111111001011011001011110011100000000000110011110001010111001101101101000100111101100111001111110011011100000000000001110010001001101111110101011101010000010101111001100111000000110100011000000000101101001001100011100110000000000110101001000000000000001000010001000001011010001110100100001111000001000000000111101000011110110000110110100010100111001011011000011000010

In [190]:
execute(code)

116672213160